In [8]:
import pandas as pd
import torch
from tqdm.notebook import tqdm

In [2]:
datasets_root = r"E:\social-bot-data\datasets\Twibot-20"
tmp_files_root = r"E:\social-bot-data\code\First-HGT-Detector\twibot-20\preprocess\tmp-files"

In [ ]:
node2id_list = pd.read_csv(rf"{datasets_root}\node2id.csv", dtype={"node_id": str,"num_id": int}) # tweets: 1-33488192, users: 33488193-33713010
node2id = {}
for row in tqdm(node2id_list.iterrows(), desc="Generate node2id dict."):
    node2id[row[1]["node_id"]] = row[1]["num_id"]

### 利用原始edge文件和node文件生成异构图中所有边和边的类型

In [3]:
edge = pd.read_csv(f"{datasets_root}\edge.csv")

follow = edge[edge["relation"] == "follow"]
friend = edge[edge["relation"] == "friend"]
post = edge[edge["relation"] == "post"]

In [ ]:
follow_src = []
follow_dst = []
friend_src = []
friend_dst = []
post_src = []
post_dst = []

for i in tqdm(follow["source_id"].tolist(), desc="follow_src"):
    follow_src.append(node2id[i])
for i in tqdm(follow["target_id"].tolist(), desc="follow_dst"):
    follow_dst.append(node2id[i])

for i in tqdm(friend["source_id"].tolist(), desc="friend_src"):
    friend_src.append(node2id[i])
for i in tqdm(friend["target_id"].tolist(), desc="friend_dst"):
    friend_dst.append(node2id[i])

for i in tqdm(post["source_id"].tolist(), desc="post_src"):
    post_src.append(node2id[i])
for i in tqdm(post["target_id"].tolist(), desc="post_dst"):
    post_dst.append(node2id[i])

In [10]:
follow_edge = torch.tensor([follow_src, follow_dst])
friend_edge = torch.tensor([friend_src, friend_dst])
post_edge = torch.tensor([post_src, post_dst])

edge_index = torch.cat((follow_edge, friend_edge, post_edge), dim=1)
edge_type = torch.cat((torch.zeros(follow_edge.size(1)), torch.ones(friend_edge.size(1)), 2 * torch.ones(post_edge.size(1))))

In [11]:
torch.save(edge_index, rf"{tmp_files_root}\edge_index.pt")
torch.save(edge_type, rf"{tmp_files_root}\edge_type.pt")